<a href="https://colab.research.google.com/github/carolynw898/STAT946Proj/blob/main/DiffuSym.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
print('Symbolic Regression with Diffusion')

Symbolic Regression with Diffusion


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import glob
import json
from torch.utils.data import Dataset, DataLoader
import re
import numpy as np
import tqdm
import random
from utils import generateDataStrEq, processDataFiles, CharDataset
from models import tNet, PointNetConfig

In [2]:
blockSize = 32
numVars = 1
numYs = 1
numPoints = 250
target = 'Skeleton'
addVars = True
const_range = [-2.1, 2.1]
trainRange = [-3.0, 3.0]
decimals = 8
embSize = 512

In [3]:
from utils import processDataFiles

# process training files from scratch
path = "0_1_0_13062021_174033.json"
files = glob.glob(path)
text = processDataFiles(files)
chars = sorted(list(set(text))+['_','T','<','>',':']) # extract unique characters from the text before converting the text to a list, # T is for the test data
text = text.split('\n') # convert the raw text to a set of examples
trainText = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(trainText) # shuffle the dataset, it's important specailly for the combined number of variables experiment
train_dataset = CharDataset(text, blockSize, chars, numVars=numVars, 
                numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                const_range=const_range, xRange=trainRange, decimals=decimals, augment=False) 

data has 967 examples, 52 unique.


In [11]:
train_dataset

In [4]:
loader = DataLoader(train_dataset, batch_size=4)

In [6]:
train_dataset.__getitem__(5)

(tensor([23, 12, 22, 25,  5, 49, 12,  5,  5, 13,  6, 25,  5, 49, 12,  6, 25, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]),
 tensor([12, 22, 25,  5, 49, 12,  5,  5, 13,  6, 25,  5, 49, 12,  6, 25, 24, 36,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]),
 tensor([[ 0.9774,  2.0756,  1.2248,  2.5588, -2.5195, -0.6410,  1.1526, -2.1722,
          -2.0410, -2.7973,  2.5072,  0.9566,  0.4874,  1.1751,  2.3139, -0.8710,
           1.3261, -2.3743, -2.7984,  1.3694,  0.1516, -2.9298,  0.7702, -1.3082,
           2.4207,  0.0958, -2.2913,  0.0600,  1.5200,  2.2456,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
           0.0000,  0.0000,  0.0000,  0.0000,  0

In [5]:
pnConfig = PointNetConfig(embeddingSize = embSize, numberofPoints = numPoints, numberofVars = numVars, numberofYs = numYs)

In [8]:
pnConfig

In [6]:
tNet_test = tNet(pnConfig)
tNet_test

tNet(
  (conv1): Conv1d(2, 512, kernel_size=(1,), stride=(1,))
  (conv2): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
  (conv3): Conv1d(1024, 2048, kernel_size=(1,), stride=(1,))
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (input_batch_norm): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [7]:
for x, y, p, v in loader:
    print(v)
    break

tensor([1, 1, 1, 1])


In [11]:
loader

In [ ]:
pbar = tqdm.tqdm(enumerate(loader), total=len(loader))  
for i, (x, y, p, v) in pbar:
    pointnet = tNet_test(p)
    pbar.set_description(f"Processing batch {i}")

  0%|          | 0/242 [00:00<?, ?it/s]

y: tensor([[12, 22, 25,  5, 39, 49, 46,  3, 25,  5, 49, 12, 10,  3, 25,  5, 49, 12,
          4,  4,  6, 25, 24, 36, 36, 36, 36, 36, 36, 36, 36, 36],
        [12, 22, 25,  5, 39, 49, 46,  3, 25,  5, 49, 12,  4,  5, 25,  5, 38, 45,
         47,  3, 25,  5, 49, 12,  5,  5, 13,  6, 25,  5, 49, 12],
        [12, 22, 25,  5, 49, 12,  5,  5, 16,  6, 25,  5, 49, 12,  5,  5, 15,  6,
         25,  5, 49, 12,  5,  5, 14,  6, 25,  5, 49, 12,  5,  5],
        [12, 22, 25,  5, 49, 12, 10,  3, 25,  5, 49, 12,  6, 25,  4,  6, 25, 24,
         36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36]]) pointnet: tensor([[0.0000, 0.7469, 0.3979,  ..., 0.0000, 0.0000, 0.8764],
        [1.7106, 0.0000, 0.0000,  ..., 1.6523, 1.6659, 0.0000],
        [0.0000, 0.0000, 0.1774,  ..., 0.0000, 0.0000, 1.0505],
        [0.0000, 1.1589, 1.0621,  ..., 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReluBackward0>)
y shape: torch.Size([4, 32]) pointnet shape: torch.Size([4, 512])


In [ ]:
class ConstantOptimizer():
    def __init__(self):
        super(self)